# Example Calculation of the Load Differences Between a Pair of Scenarios


In [ ]:
# Start by importing the packages we need:
import os

import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


## Set the Directory Structure

In [ ]:
# Identify the data input and image output directories:
data_input_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/outputs/tell_output/'
image_output_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2024_applied_energy/figures/'


## Process the Data


In [ ]:
# Read in the state-level time series for 'rcp85hotter_ssp5':
df_one = pd.read_csv(data_input_dir + 'rcp85hotter_ssp5/2080/TELL_Balancing_Authority_Hourly_Load_Data_2080_Scaled_2080.csv')

# Rename a few columns for simplicity:
df_one.rename(columns={'Scaled_TELL_BA_Load_MWh': 'Load_One_MWh'}, inplace=True)

# Only keep the columns that are needed:
df_one = df_one[['BA_Code', 'Time_UTC', 'Load_One_MWh']].copy()

# Read in the state-level time series for 'rcp45hotter_ssp5':
df_two = pd.read_csv(data_input_dir + 'rcp85cooler_ssp5/2080/TELL_Balancing_Authority_Hourly_Load_Data_2080_Scaled_2080.csv')
            
# Rename a few columns for simplicity:
df_two.rename(columns={'Scaled_TELL_BA_Load_MWh': 'Load_Two_MWh'}, inplace=True)
       
# Only keep the columns that are needed:
df_two = df_two[['BA_Code', 'Time_UTC', 'Load_Two_MWh']].copy()
        
# Merge the two time series together on state name and time:
merged_df = df_one.merge(df_two, on=['BA_Code', 'Time_UTC'])
            
# Calculate the mean load between the two time-series as well as the absolute and relative load biases:
merged_df['Mean_Load_MWh'] =  merged_df[['Load_One_MWh', 'Load_Two_MWh']].mean(axis=1)
merged_df['Absolute_Difference_MWh'] = abs(merged_df['Load_One_MWh'] - merged_df['Load_Two_MWh'])
merged_df['Relative_Difference_%'] = 100*(merged_df['Absolute_Difference_MWh']/merged_df['Mean_Load_MWh']).round(4)

# Convert the time to a datetime variable:
merged_df['Time_UTC'] = pd.to_datetime(merged_df['Time_UTC'], format='%Y-%m-%d %H:%M:%S')

# Subset to just the data for CISO:
merged_df = merged_df[merged_df['BA_Code'] == 'CISO'].copy()

# Subset to the top 10% of all mean load hours:
peak_df = merged_df.nlargest(876,'Mean_Load_MWh')

# Compute the mean absolute and relative differences for the full dataset and peak subset:
merged_df['Mean_Abs_Diff_MWh'] = merged_df['Absolute_Difference_MWh'].mean().round(2)
merged_df['Mean_Rel_Diff_%'] = merged_df['Relative_Difference_%'].mean().round(2)
merged_df['Peak_Abs_Diff_M1_MWh'] = peak_df['Absolute_Difference_MWh'].mean().round(2)
merged_df['Peak_Rel_Diff_M1_%'] = peak_df['Relative_Difference_%'].mean().round(2)
#merged_df['Peak_Abs_Diff_M2_MWh'] = (abs(merged_df['Load_One_MWh'].max() - merged_df['Load_Two_MWh'].max())).round(2)
merged_df['Peak_Abs_Diff_M2_MWh'] = (abs(merged_df['Load_One_MWh'].max() - merged_df['Load_Two_MWh'].max()))
merged_df['Peak_Rel_Diff_M2_%'] = 100*(merged_df['Peak_Abs_Diff_M2_MWh'] / ((merged_df['Load_One_MWh'].max() + merged_df['Load_Two_MWh'].max())/2)).round(2)

merged_df


## Make the Plot


In [ ]:
# Make the plot:
plt.figure(figsize=(24, 20))
plt.rcParams['font.size'] = 16
    
plt.subplot(311)
plt.plot(merged_df['Time_UTC'], merged_df['Load_One_MWh'], color='r', linestyle='-', label='rcp85hotter_ssp5', linewidth=1)
plt.plot(merged_df['Time_UTC'], merged_df['Load_Two_MWh'], color='b', linestyle='-', label='rcp85cooler_ssp5', linewidth=1)
plt.legend(loc='upper left', prop={'size': 14})
plt.xlabel('')
plt.ylabel('Hourly Load [MWh]')
plt.title('CISO Load Time Series in 2080')
plt.title('a)', loc='left', fontsize=16)

plt.subplot(312)
plt.plot(merged_df['Time_UTC'], merged_df['Absolute_Difference_MWh'], color='c', linestyle='-', label='rcp85hotter_ssp5 - rcp85cooler_ssp5', linewidth=1)
plt.plot(merged_df['Time_UTC'], merged_df['Mean_Abs_Diff_MWh'], color='k', linestyle='-', label='Mean Load Difference', linewidth=1)
plt.plot(merged_df['Time_UTC'], merged_df['Peak_Abs_Diff_M2_MWh'], color='gray', linestyle='-', label='Peak Load Difference', linewidth=1)
plt.legend(loc='upper left', prop={'size': 14})
plt.xlabel('') 
plt.ylabel('Absolute Difference [MWh]')
plt.title('Absolute Difference in Demand')
plt.title('b)', loc='left', fontsize=16)

plt.subplot(313)
plt.plot(merged_df['Time_UTC'], merged_df['Relative_Difference_%'], color='m', linestyle='-', label='rcp85hotter_ssp5 - rcp85cooler_ssp5', linewidth=1)
plt.plot(merged_df['Time_UTC'], merged_df['Mean_Rel_Diff_%'], color='k', linestyle='-', label='Mean Load Difference', linewidth=1)
plt.plot(merged_df['Time_UTC'], merged_df['Peak_Rel_Diff_M2_%'], color='gray', linestyle='-', label='Peak Load Difference', linewidth=1)
plt.legend(loc='upper left', prop={'size': 14})
plt.xlabel('') 
plt.ylabel('Relative Difference [%]')
plt.title('Relative Difference in Demand ')
plt.title('c)', loc='left', fontsize=16)

filename = ('Calculations.png')
plt.savefig(os.path.join(image_output_dir, filename), dpi=300, bbox_inches='tight', facecolor='white')


In [ ]:
pd.to_datetime('2080-08-10 00:00:00')


In [ ]:
# Make the plot:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=[24, 20])
plt.rcParams['font.size'] = 18

axins1 = inset_axes(ax1, width="35%", height="40%", loc=2, borderpad=0.6)
axins2 = inset_axes(ax2, width="35%", height="40%", loc=2, borderpad=0.6)
axins3 = inset_axes(ax3, width="35%", height="40%", loc=2, borderpad=0.6)

ax1.plot(merged_df['Time_UTC'], merged_df['Load_One_MWh'], color='r', linestyle='-', label='rcp85hotter_ssp5', linewidth=1)
ax1.plot(merged_df['Time_UTC'], merged_df['Load_Two_MWh'], color='b', linestyle='-', label='rcp85cooler_ssp5', linewidth=1)
ax1.plot([pd.to_datetime('2080-08-11 12:00:00'), pd.to_datetime('2080-08-11 12:00:00')], [38000, 112000], color='k', linestyle='--', label='', linewidth=3)
ax1.plot([pd.to_datetime('2080-08-17 12:00:00'), pd.to_datetime('2080-08-17 12:00:00')], [38000, 112000], color='k', linestyle='--', label='', linewidth=3)
axins1.plot(merged_df['Time_UTC'], merged_df['Load_One_MWh'], color='r', linestyle='-', label='rcp85hotter_ssp5', linewidth=1)
axins1.plot(merged_df['Time_UTC'], merged_df['Load_Two_MWh'], color='b', linestyle='-', label='rcp85cooler_ssp5', linewidth=1)
ax1.legend(loc='upper right', prop={'size': 14})
ax1.set(xlim=(pd.to_datetime(merged_df['Time_UTC'].min()), pd.to_datetime(merged_df['Time_UTC'].max())), ylim=(38000, 112000))
axins1.set(xlim=(pd.to_datetime('2080-08-11 12:00:00'), pd.to_datetime('2080-08-17 12:00:00')), ylim=(38000, 112000))
ax1.set_xlabel('')
ax1.set_ylabel('Hourly Load [MWh]')
ax1.set_title('CISO Load Time Series in 2080')
ax1.set_title('a)', loc='left', fontsize=18)

ax2.plot(merged_df['Time_UTC'], merged_df['Absolute_Difference_MWh'], color='c', linestyle='-', label='rcp85hotter_ssp5 - rcp85cooler_ssp5', linewidth=1)
ax2.plot(merged_df['Time_UTC'], merged_df['Mean_Abs_Diff_MWh'], color='k', linestyle='-', label='Mean Load Difference', linewidth=2)
ax2.plot(merged_df['Time_UTC'], merged_df['Peak_Abs_Diff_M2_MWh'], color='gray', linestyle='-', label='Peak Load Difference', linewidth=2)
ax2.plot([pd.to_datetime('2080-08-11 12:00:00'), pd.to_datetime('2080-08-11 12:00:00')], [0, 7500], color='k', linestyle='--', label='', linewidth=3)
ax2.plot([pd.to_datetime('2080-08-17 12:00:00'), pd.to_datetime('2080-08-17 12:00:00')], [0, 7500], color='k', linestyle='--', label='', linewidth=3)
axins2.plot(merged_df['Time_UTC'], merged_df['Absolute_Difference_MWh'], color='c', linestyle='-', label='rcp85hotter_ssp5 - rcp85cooler_ssp5', linewidth=1)
ax2.legend(loc='upper right', prop={'size': 14})
ax2.set(xlim=(pd.to_datetime(merged_df['Time_UTC'].min()), pd.to_datetime(merged_df['Time_UTC'].max())), ylim=(0, 7500))
axins2.set(xlim=(pd.to_datetime('2080-08-11 12:00:00'), pd.to_datetime('2080-08-17 12:00:00')), ylim=(0, 7500))
ax2.set_xlabel('') 
ax2.set_ylabel('Absolute Difference [MWh]')
ax2.set_title('Absolute Difference in Demand')
ax2.set_title('b)', loc='left', fontsize=18)

ax3.plot(merged_df['Time_UTC'], merged_df['Relative_Difference_%'], color='m', linestyle='-', label='rcp85hotter_ssp5 - rcp85cooler_ssp5', linewidth=1)
ax3.plot(merged_df['Time_UTC'], merged_df['Mean_Rel_Diff_%'], color='k', linestyle='-', label='Mean Load Difference', linewidth=2)
ax3.plot(merged_df['Time_UTC'], merged_df['Peak_Rel_Diff_M2_%'], color='gray', linestyle='-', label='Peak Load Difference', linewidth=2)
ax3.plot([pd.to_datetime('2080-08-11 12:00:00'), pd.to_datetime('2080-08-11 12:00:00')], [0, 11], color='k', linestyle='--', label='', linewidth=3)
ax3.plot([pd.to_datetime('2080-08-17 12:00:00'), pd.to_datetime('2080-08-17 12:00:00')], [0, 11], color='k', linestyle='--', label='', linewidth=3)
axins3.plot(merged_df['Time_UTC'], merged_df['Relative_Difference_%'], color='m', linestyle='-', label='rcp85hotter_ssp5 - rcp85cooler_ssp5', linewidth=1)
ax3.legend(loc='upper right', prop={'size': 14})
ax3.set(xlim=(pd.to_datetime(merged_df['Time_UTC'].min()), pd.to_datetime(merged_df['Time_UTC'].max())), ylim=(0, 11))
axins3.set(xlim=(pd.to_datetime('2080-08-11 12:00:00'), pd.to_datetime('2080-08-17 12:00:00')), ylim=(0, 11))
ax3.set_xlabel('') 
ax3.set_ylabel('Relative Difference [%]')
ax3.set_title('Relative Difference in Demand ')
ax3.set_title('c)', loc='left', fontsize=18)

for axi in [axins1, axins2, axins3]:
    axi.tick_params(labelleft=False, labelbottom=False)

filename = ('Calculations.png')
plt.savefig(os.path.join(image_output_dir, filename), dpi=300, bbox_inches='tight', facecolor='white')
